In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

print('importing libraries..')

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import cv2
import pandas as pd
import difflib
import time
import plotly.plotly as py
import plotly.graph_objs as go
from skimage.measure import compare_ssim as ssim
from tqdm import tqdm_notebook

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

sys.path.insert(0, os.path.abspath(".."))
from utils import label_map_util
from utils import visualization_utils as vis_util

# MODEL_NAME = 'faster_rcnn_inception_resnet_v2_atrous_coco_11_06_2017'
MODEL_NAME = 'faster_rcnn_resnet101_coco_11_06_2017'

PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

print ('loading model..')
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
print('model loaded')

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

importing libraries..
loading model..
model loaded


## Human Detection Process

In [ ]:
def detection(path):
    points_objs = {}
    frame_objs = {}
    start = time.time()
    
    with detection_graph.as_default():
        with tf.Session(graph=detection_graph) as sess:
            skip = 0
            cap = cv2.VideoCapture(path)
            id_frame = 1
            id_center = 0
            
            while(True):
                ret, frame = cap.read()
                if cv2.waitKey(1) & 0xFF == ord('q'): break
                skip = 0
                image_np = np.array(frame)
                if(image_np.shape == ()): break

                print('Frame ID:', id_frame, '\tTime:', '{0:.2f}'.format(time.time()-start), 'seconds')
                frame_objs[id_frame] = image_np
                image_np_expanded = np.expand_dims(image_np, axis=0)


                image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
                boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
                scores = detection_graph.get_tensor_by_name('detection_scores:0')
                classes = detection_graph.get_tensor_by_name('detection_classes:0')
                num_detections = detection_graph.get_tensor_by_name('num_detections:0')

                (boxes, scores, classes, num_detections) = sess.run([boxes, scores, classes, num_detections],
                    feed_dict={image_tensor: image_np_expanded})

                boxes = np.squeeze(boxes)
                classes = np.squeeze(classes).astype(np.int32)
                scores = np.squeeze(scores)

                count_boxes = 0
                thresh = 0.3
                max_boxes = 50

                for i, c in enumerate(classes):
                    if (c == 1 and (scores[i] > thresh) and (count_boxes < max_boxes)):
                        im_height = image_np.shape[0]
                        im_width = image_np.shape[1]
                        ymin, xmin, ymax, xmax = boxes[i]
                      
                        (left, right, top, bottom) = (int(xmin*im_width),  int(xmax*im_width),
                                                      int(ymin*im_height), int(ymax*im_height))
                        points_objs[id_center] = {
                            'frame': id_frame,
                            'bbox': [left, top, right-left, bottom-top],
                            'score': scores[i]
                        }
                        count_boxes += 1

                id_center += 1
                id_frame += 1
                
            cap.release()
            cv2.destroyAllWindows()

        return points_objs, frame_objs

## Testing

In [ ]:
points_objs, frame_objs = detection(path='../data/1_BEST/1_BEST.mp4')

Frame ID: 0 	Time: 0.35 seconds
Frame ID: 1 	Time: 11.38 seconds
Frame ID: 2 	Time: 12.73 seconds
Frame ID: 3 	Time: 14.08 seconds
Frame ID: 4 	Time: 15.42 seconds
Frame ID: 5 	Time: 16.75 seconds
Frame ID: 6 	Time: 18.10 seconds
Frame ID: 7 	Time: 19.45 seconds
Frame ID: 8 	Time: 20.80 seconds
Frame ID: 9 	Time: 22.16 seconds
Frame ID: 10 	Time: 23.52 seconds
Frame ID: 11 	Time: 24.87 seconds
Frame ID: 12 	Time: 26.24 seconds
Frame ID: 13 	Time: 27.60 seconds
Frame ID: 14 	Time: 28.96 seconds
Frame ID: 15 	Time: 30.33 seconds
Frame ID: 16 	Time: 31.71 seconds
Frame ID: 17 	Time: 33.07 seconds
Frame ID: 18 	Time: 34.42 seconds
Frame ID: 19 	Time: 35.78 seconds
Frame ID: 20 	Time: 37.13 seconds
Frame ID: 21 	Time: 38.48 seconds
Frame ID: 22 	Time: 39.84 seconds
Frame ID: 23 	Time: 41.19 seconds
Frame ID: 24 	Time: 42.54 seconds
Frame ID: 25 	Time: 43.93 seconds
Frame ID: 26 	Time: 45.28 seconds
Frame ID: 27 	Time: 46.63 seconds
Frame ID: 28 	Time: 47.98 seconds
Frame ID: 29 	Time: 49.33

In [ ]:
df_points = pd.DataFrame.from_dict(points_objs, orient='index')
df_points.head()

In [ ]:
df_boxes = []
for left,top,width,height in df_points['bbox'].as_matrix():
    df_boxes.append([left,top,width,height])
df_boxes = pd.DataFrame.from_records(df_boxes)
df_boxes.head()

In [ ]:
df_minus_ones = pd.DataFrame.from_records([[-1] for x in range(len(df_boxes))])
df_minus_ones.head()

In [ ]:
df_MOT = pd.concat([df_points['frame'], 
                    df_minus_ones, 
                    df_boxes, 
                    df_points['score'], 
                    df_minus_ones, 
                    df_minus_ones, 
                    df_minus_ones], axis=1)
df_MOT.head(10)

In [ ]:
df_MOT.to_csv('../deep_sort/MOT15/train/PETS09-S2L1/det/det.txt', header=None, index=False)